# Project 3 Part 4
 Nokuthula Mchunu

In [177]:
# Importing Relevent Liberies 
import os, time,json
import tmdbsimple as tmdb 
import pandas as pd
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/Data"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['tmdb_data_2001.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints',
 'Data']

In [178]:
import json
with open('/Users/noksmchunu/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()


dict_keys(['tmdb_api_key', 'tmdb_access_token'])

In [179]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['tmdb_api_key']


In [180]:
basics  = pd.read_csv("/Users/noksmchunu/Downloads/title_basics.csv")
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0070596,movie,Socialist Realism,El realismo socialista,0,2023,NaN,78,Drama


In [181]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

def get_movie_with_rating (movie_id):
    
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies (movie_id)
    # save the info releases dictionaries
    info = movie.info ()
    releases = movie.releases ( )
    # Loop through countries in releases
    for c in releases[ 'countries']:
        # if the country abbreviation==Us
        if c['iso_3166_1'] =='US':
            ## save a "certification" key in info with the certification
            info ['certification'] = c['certification']
    return info

Part 4: Use an API to extract ADDITIONAL box office revenue and profit data to add to your IMDB data and perform exploratory data analysis.

In [182]:
# Downloading the more movies 
def download_movies():
    Years_to_get  = [2014,2015,2016,2017,2018,2019,2020]
    errors = []
    # Start of OUTER loop
    for YEAR in tqdm_notebook(Years_to_get, desc='YEARS', position=0):
        #Defining the JSON file to store results for year
        JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
        # Check if file exists
        file_exists = os.path.isfile(JSON_FILE)
        # If it does not exist: create it
        if file_exists == False:
            # save an empty dict with just "imdb_id" to the new json file.
            with open(JSON_FILE,'w') as f:
                json.dump([{'imdb_id':0}],f)
                #Saving new year as the current df
        df = basics.loc[ basics['startYear']==YEAR].copy()
        # saving movie ids to list
        movie_ids = df['tconst'].copy()
        # Load existing data from json into a dataframe called "previous_df"
        previous_df = pd.read_json(JSON_FILE)
        # filter out any ids that are already in the JSON_FILE
        movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
        #Get index and movie id from list
        # INNER Loop
        for movie_id in tqdm_notebook(movie_ids_to_get,
                                      desc=f'Movies from {YEAR}',
                                      position=1,
                                      leave=True):
            try:
                # Retrieve then data for the movie id
                temp = get_movie_with_rating(movie_id)  
                # Append/extend results to existing file using a pre-made function
                write_json(temp,JSON_FILE)
                # Short 20 ms sleep to prevent overwhelming server
                time.sleep(0.02)
                
            except Exception as e:

                

                
                errors.append([movie_id, e])
                       
    final_year_df = pd.read_json(JSON_FILE) 
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)
    print(f"- Total errors: {len(errors)}")







In [ ]:
# calling the download_movie function 
download_movies()

YEARS:   0%|          | 0/7 [00:00<?, ?it/s]

Movies from 2014:   0%|          | 0/3534 [00:00<?, ?it/s]